In [204]:
import os
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer

In [205]:
os.listdir(os.getcwd()+"\\training-v1")

['offenseval-annotation.txt',
 'offenseval-training-v1.tsv',
 'readme-trainingset-v1.txt']

In [206]:
train_data = []
train_df = pd.DataFrame()

with open('.\\training-v1\\offenseval-training-v1.tsv','r', encoding = 'utf-8') as in_file:
    #train_data = [line.strip().split('\t') for line in in_file]
    
    for line in in_file:
        train_df = pd.concat([train_df,pd.DataFrame([line.strip().split('\t')])], axis = 0)
        
train_df.reset_index(inplace=True)

### Analyzing the data and response variables

In [16]:
train_df.columns = train_df.iloc[0,:].values
train_df = train_df.iloc[1:,]

In [17]:
train_df.shape

(13240, 5)

In [18]:
train_df.subtask_a.value_counts()

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64

In [19]:
train_df.subtask_b.value_counts()

NULL    8840
TIN     3876
UNT      524
Name: subtask_b, dtype: int64

In [20]:
train_df.subtask_c.value_counts()

NULL    9364
IND     2407
GRP     1074
OTH      395
Name: subtask_c, dtype: int64

In [21]:
train_df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NULL
0,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
0,16820,Amazon is investigating Chinese employees who ...,NOT,NULL,NULL
0,62688,"""@USER Someone should'veTaken"""" this piece of ...",OFF,UNT,NULL
0,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NULL,NULL


### Feature Engineering

#### Sentiment Polarity

In [73]:
sentiment_polarity_values = train_df.tweet.apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x))

In [89]:
polarity_array = []
np.ext([1,2,3],[1,2,3])

array([1, 2, 3, 1, 2, 3])

In [94]:
polarity_array = []

for each in sentiment_polarity_values:
    temp_polarity_array = list(each.values())
    polarity_array.append(temp_polarity_array)

### Feature Engineering

In [202]:
polarity_array = np.array()

for each in range(sentiment_polarity_vallues):
    print(each)

TypeError: Required argument 'object' (pos 1) not found

In [74]:
# for each in range(len(x)):
#     print(x[each].values())

#### Bag of Words

In [140]:
#stop_words=stopwords.words('english')
bow_counts = CountVectorizer(tokenizer = word_tokenize,ngram_range=(1,2))
bow_data = bow_counts.fit_transform(train_df.tweet)

In [193]:
bow_data

<13240x156910 sparse matrix of type '<class 'numpy.int64'>'
	with 608756 stored elements in Compressed Sparse Row format>

#### Tf Idf Vectorizer

In [24]:
tfidf_counts = CountVectorizer(tokenizer = word_tokenize,ngram_range=(1,2))
tfidf_data = tfidf_counts.fit_transform(train_df.tweet)

#### GloVe Embeddings

In [195]:
from gensim.models import Word2Vec

# model = Word2Vec(common_texts, size=200, window=5, min_count=1, workers=4)
# model.save("word2vec.model")

#model = Word2Vec.load_word2vec_format(os.path.join(os.path.dirname(__file__), 'GoogleNews-vectors-negative300.bin'), binary=True)

In [194]:
#bow_data.head()

In [ ]:
model = Word2Vec(common_texts, size=200, window=5, min_count=1, workers=4)
#model.save("word2vec.model")


In [ ]:
model.most_similar('dog')

In [115]:
model = Word2Vec.load("word2vec.model")

In [116]:
model.train([["hello", "world"]], total_examples=1, epochs=1)

(0, 2)

In [132]:
#list(model.vocabulary)

In [120]:
vector = model.wv['computer'] 
vector

array([-1.9362402e-03, -1.8159261e-03, -2.3032348e-03,  1.2677729e-03,
       -1.3240631e-04, -3.5193078e-03, -2.7348676e-03,  3.8732493e-03,
       -4.8462278e-03, -4.1700766e-04, -3.5477204e-03,  4.8351856e-03,
       -4.0507466e-03, -3.9417990e-03, -3.7672587e-03, -3.5563539e-04,
       -2.2007199e-03, -2.8392107e-03,  9.2029892e-04,  2.1450862e-03,
        7.0885639e-04, -3.1044749e-03, -4.7155614e-03, -1.2815850e-04,
        2.3153522e-03,  3.9976821e-03, -4.8585776e-03, -2.1720717e-03,
       -4.9299761e-03, -2.4226194e-03, -2.7628355e-03,  1.5553590e-03,
       -3.5330412e-04, -1.9692027e-03,  4.3059010e-03,  1.2162674e-03,
       -4.4145077e-03,  9.7441569e-04,  2.0901181e-03,  2.7674863e-03,
        4.2085508e-03,  4.7730752e-03, -3.9273798e-03,  1.2004472e-03,
       -3.6889508e-03, -8.6263189e-04,  3.6402507e-05,  1.9060946e-03,
       -4.2189821e-03,  1.6854763e-03,  7.2570430e-04,  8.9747051e-04,
       -4.1444935e-03, -2.5958517e-03,  1.0577564e-03,  2.2620117e-04,
      

### Feature Engineering

### Feature Engineering

#### POS Tagging

#### Dependency Parsing

### Train test split

In [25]:
#senti_analyser = SentimentAnalyzer()
#senti_analyser.all_words(train_df.tweet.values[0])

In [26]:
X_train, X_test, y_train, y_test =  train_test_split(bow_data,train_df.subtask_a,test_size = 0.3)

### Model Creation

#### Logistic Regression Model

In [27]:
for C in [0.75,1,10,50,100]:
    lr_model = LogisticRegression(C = C)
    lr_model.fit(X_train,y_train)
    test_pred = lr_model.predict(X_test)

    #acc = 100 * np.sum(test_pred == y_test)/len(y_test)
    print(C,": ",f1_score(y_test,test_pred,average='weighted') )

0.75 :  0.7405156645908003
1 :  0.7410220250787326
10 :  0.7430990980508277
50 :  0.7434590177652464
100 :  0.744373841112178


In [34]:
(y_test == test_pred).sum()/len(y_test)

0.7535246727089627

In [31]:
y_test.value_counts()

NOT    2661
OFF    1311
Name: subtask_a, dtype: int64

In [33]:
674 + 637

1311

In [30]:
confusion_matrix(y_test,test_pred)

array([[2319,  342],
       [ 637,  674]], dtype=int64)

In [137]:
lr_model = LogisticRegression(C = 0.75)
lr_model.fit(X_train,y_train)
test_pred = lr_model.predict(X_test)

print(C,": ",100 * np.sum(test_pred == y_test)/len(y_test))
confusion_matrix(y_test,test_pred)

100 :  71.82779456193353


array([[2632,   30],
       [1089,  221]], dtype=int64)

#### Random Forest

In [138]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)

test_pred = lr_model.predict(X_test)
f1_score(y_test,test_pred,average='weighted') 

0.646086411710712

#### Top Features